## FeS - Trying to get the HF energy correct

In [1]:
import basis_set_exchange as bse

basis_dict = bse.get_basis('ANO-RCC-MB', elements=['Fe', 'S'], fmt='nwchem')

In [2]:
import numpy as np
from pyscf import gto, scf, mcscf
from pyscf import gto, ao2mo, scf
import pennylane as qml

# Your original setup (unchanged)
mol_pyscf = gto.M(
    atom = 'Fe 0 0 0; S 0 0 1.826',
    basis = basis_dict,
    spin = 4)
rhf = scf.UHF(mol_pyscf).newton()
rhf.max_cycle_inner = 100 
#rhf.max_cycle = 1000 
rhf.kernel()


#Newton based method
#mf = scf.RHF(mol).newton()

#print(mol_pyscf.nelec)  # Print the atom information to verify

print("Atom list:", mol_pyscf.atom)
#print("Basis used:", mol_pyscf.basis)
print("Charge:", mol_pyscf.charge)
print("Spin:", mol_pyscf.spin)
print("Number of electrons:", mol_pyscf.nelectron)

# === ACTIVE SPACE IMPLEMENTATION ===
# Step 1: Define active space parameters
norb_active = 6      # Number of active orbitals
nelec_active = 6     # Number of active electrons

# Step 2: Set up CASSCF to get active space integrals
mycas = mcscf.CASSCF(rhf, norb_active, nelec_active)

# Step 3: Get effective active space integrals
h1e_cas, ecore = mycas.get_h1eff()  # One-electron integrals + core energy
h2e_cas = mycas.get_h2eff()         # Two-electron integrals

print("h2e_cas type:", type(h2e_cas))
print("h2e_cas shape:", h2e_cas.shape) # This result shows a 2D array, symmetric reduced form

norb = norb_active  

# "21" = norb*(norb+1)//2 for norb=6, so s4 symmetry - It was symmetrical reduced. Got into case with no symmetry
h2e_4d = ao2mo.restore(1, h2e_cas, norb) #Check this link (https://pyscf.org/pyscf_api_docs/pyscf.ao2mo.html)




one_mo = h1e_cas
two_mo = h2e_4d
h2_phys = np.swapaxes(two_mo, 1, 3)  # Ensure the two-electron integrals are in the correct order
core_constant = np.array([ecore])

# Step 5: Continue with your original PennyLane code (unchanged)
H_fermionic = qml.qchem.fermionic_observable(core_constant, one_mo, h2_phys)
H = qml.jordan_wigner(H_fermionic)


converged SCF energy = -1655.10155371148  <S^2> = 6.0386344  2S+1 = 5.01543
Atom list: Fe 0 0 0; S 0 0 1.826
Charge: 0
Spin: 4
Number of electrons: 42
h2e_cas type: <class 'numpy.ndarray'>
h2e_cas shape: (21, 21)


## Trying to restart using chk file

In [9]:
from pyscf import gto, scf
from pyscf import scf, scf
from pyscf.scf.stability import uhf_stability as stability

# (Re-)define your molecule as usual
mol = gto.M(atom= 'Fe 0 0 0; S 0 0 1.826', basis= basis_dict,  spin=4)
mol.build()

mf = scf.UHF(mol)  # or scf.RHF as appropriate
mf.chkfile = 'FeS.chk'
mf.kernel()  # This run will produce the chkfile


mo = stability(mf, internal=True, external=False , return_status=True)

mf = mf.run(mo_coeff=mo)


converged SCF energy = -1655.1015537099  <S^2> = 6.0386361  2S+1 = 5.0154306
<class 'pyscf.scf.uhf.UHF'> wavefunction has an internal instability.
mo ((array([[ 9.99956373e-01,  3.27733275e-06, -8.68591199e-03,
         7.87612006e-08, -5.11328704e-08,  1.62224992e-05,
        -2.27258761e-05, -1.46055689e-07, -1.12415229e-07,
         1.48383225e-05,  3.03521144e-03,  4.37551402e-07,
         4.40368360e-07, -1.54480320e-04, -2.76518794e-03,
        -1.55367686e-06, -2.53082343e-06, -1.78526580e-06,
        -1.41405236e-03,  6.89733991e-07,  6.57688311e-07,
         4.26160829e-03,  1.76956068e-03,  1.04216021e-03,
         5.21969677e-07,  1.32958728e-06,  5.49233211e-03],
       [ 8.72375870e-03,  2.41507925e-05,  9.99878429e-01,
         5.78900788e-07, -3.76271633e-07, -1.50489149e-03,
        -1.69242133e-04, -1.08894180e-06, -8.35222480e-07,
         1.29341580e-04, -1.38524988e-02,  3.30168870e-06,
         3.33869269e-06, -4.19935495e-04, -1.74948751e-02,
        -1.12233904e-

## Trying stability

In [4]:
from pyscf import gto, scf
from pyscf.scf.stability import uhf_stability

mol = gto.Mole()
mol.atom = '''
Fe 0 0 0
S 0 0 1.826
'''
mol.basis = basis_dict
mol.spin = 4
mol.charge = 0
mol.build()



mf = scf.UHF(mol)
mf.max_cycle = 100
mf.conv_tol = 1e-8
mf.conv_tol_grad = 1e-5
mf.verbose = 4
mf.kernel()

# Check stability
stability_mo = uhf_stability(mf, internal=True, external=False, verbose=4)

# If instability found, restart SCF with new stable MOs
if stability_mo is not None:
    mf.kernel(mo_coeff=stability_mo)




******** <class 'pyscf.scf.uhf.UHF'> ********
method = UHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-08
SCF conv_tol_grad = 1e-05
SCF max_cycles = 100
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /var/folders/q1/krfdq3k10jlff3t1x7mqtv6r0000gn/T/tmprhd53kf5
max_memory 4000 MB (current use 0 MB)
number electrons alpha = 23  beta = 19
init E= -1654.2953238867

WARN: alpha nocc = 23  HOMO 0.384027347442066 >= LUMO 0.3840273474421

  beta  nocc = 19  HOMO = 0.163764665554238  LUMO = 0.341176367707052

WARN: system HOMO 0.384027347442066 >= system LUMO 0.341176367707052

cycle= 1 E= -1651.07844714194  delta_E= 3.22  |g|= 2.11  |ddm|=  3.9
  alpha nocc = 23  HOMO = -0.608213483558423  LUMO = -0.389479774635918
  beta  nocc = 19  HOMO = -1.74276346346601  LUMO = -1.0462351016768

WARN: system HOMO -0.431415364109424 >= system LUMO -1.0462351

## Restricted HF

In [10]:
from pyscf import gto, scf
from pyscf.scf.stability import rohf_stability

# Build your molecule
mol = gto.Mole()
mol.atom = 'Fe 0 0 0; S 0 0 1.826'
mol.basis = basis_dict  # Replace with your desired basis
mol.spin = 4
mol.charge = 0
mol.build()

# Perform UHF calculation
mf = scf.ROHF(mol)
#mf = scf.newton(mf)
mf.kernel()

# Check internal stability of the HF solution
from pyscf.scf.stability import uhf_stability

max_iter = 5
for i in range(max_iter):
    stability_mo = rohf_stability(mf, internal=True, external=False, verbose=4)
    if stability_mo is None:
        print(f"Wavefunction is stable after {i} iterations.")
        break
    else:
        print(f"Internal instability detected at iteration {i}. Restarting SCF ...")
        mf.kernel(mo_coeff=stability_mo)
else:
    print("Warning: Could not reach stability after multiple attempts.")



converged SCF energy = -1655.34968383623
<class 'pyscf.scf.rohf.ROHF'> wavefunction is stable in the internal stability analysis
Internal instability detected at iteration 0. Restarting SCF ...
converged SCF energy = -1655.34968383623
<class 'pyscf.scf.rohf.ROHF'> wavefunction is stable in the internal stability analysis
Internal instability detected at iteration 1. Restarting SCF ...
converged SCF energy = -1655.34968383623
<class 'pyscf.scf.rohf.ROHF'> wavefunction is stable in the internal stability analysis
Internal instability detected at iteration 2. Restarting SCF ...
converged SCF energy = -1655.34968383623
<class 'pyscf.scf.rohf.ROHF'> wavefunction is stable in the internal stability analysis
Internal instability detected at iteration 3. Restarting SCF ...
converged SCF energy = -1655.34968383623
<class 'pyscf.scf.rohf.ROHF'> wavefunction is stable in the internal stability analysis
Internal instability detected at iteration 4. Restarting SCF ...
converged SCF energy = -1655.3

In [5]:
from pyscf import gto, scf
from pyscf.scf.stability import uhf_stability

mol = gto.Mole()
mol.atom = '''
Fe 0 0 0
S 0 0 1.826
'''
mol.basis = basis_dict
mol.spin = 4
mol.charge = 0
mol.build()

mf = scf.UHF(mol)
mf.max_cycle = 100
mf.conv_tol = 1e-8
mf.kernel()

# Switch to Newton solver if unstable
mf = scf.newton(mf)
mf.kernel()

# Iteratively check stability and restart
max_iter = 5
for i in range(max_iter):
    mo_stable = uhf_stability(mf, internal=True, external=False, verbose=4)
    if mo_stable is None:
        print(f"Wavefunction stable at iteration {i}")
        break
    else:
        print(f"Restarting SCF with stable orbitals, iteration {i}")
        mf.kernel(mo_coeff=mo_stable)
else:
    print("Could not get stable wavefunction after maximum restarts.")


converged SCF energy = -1655.10155370988  <S^2> = 6.038637  2S+1 = 5.015431
converged SCF energy = -1655.10155370989  <S^2> = 6.0386363  2S+1 = 5.0154307
<class 'pyscf.soscf.newton_ah.SecondOrderUHF'> wavefunction has an internal instability.
Restarting SCF with stable orbitals, iteration 0

WARN: Newton solver expects mo_coeff with mo_occ as initial guess but mo_occ is not found in the arguments.
      The given argument is treated as density matrix.



ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

## Using newton method

In [15]:
from pyscf import gto, scf
from pyscf.scf.stability import rohf_stability

mol = gto.Mole()
mol.atom = '''
Fe 0 0 0
S 0 0 1.826
'''
mol.basis = basis_dict
mol.spin = 4
mol.charge = 0
mol.build()

mf = scf.ROHF(mol)
mf.kernel()
#mf.init_guess = 'vsap'
# Switch to Newton solver for improved convergence
mf_newton = scf.newton(mf)
print("Starting Newton method for UHF...")
mf_newton.kernel()

# Check stability on the UHF object, not the Newton object
mo_stable = rohf_stability(mf, internal=True, external=False, verbose=4)

if mo_stable is not None:
    print("Restarting UHF with stable orbitals...")
    mf.kernel(mo_coeff=mo_stable)
    # Optional: rerun Newton with the new stable guess
    mf_newton = scf.newton(mf)
    mf_newton.kernel()

print("Final stable HF energy:", mf_newton.e_tot)


converged SCF energy = -1655.34968383623
Starting Newton method for UHF...
converged SCF energy = -1655.3496838382
<class 'pyscf.scf.rohf.ROHF'> wavefunction is stable in the internal stability analysis
Restarting UHF with stable orbitals...
converged SCF energy = -1655.34968383623
converged SCF energy = -1655.3496838382
Final stable HF energy: -1655.3496838381984


In [16]:
print(type(mf_newton))


<class 'pyscf.soscf.newton_ah.SecondOrderROHF'>


In [17]:
mol = gto.M(atom='Fe 0 0 0; S 0 0 1.826', basis=basis_dict, spin=4)
mf = scf.ROHF(mol).run(conv_tol=1e-2)
mf = scf.newton(mf).set(conv_tol=1e-12)
mf.kernel()
#Correct answer : energy = -1655.3636618836

converged SCF energy = -1654.95704457756
converged SCF energy = -1655.34948809874


np.float64(-1655.3494880987414)

## Some newton method in literature

In [3]:
#!/usr/bin/env python
#
# Author: Qiming Sun <osirpt.sun@gmail.com>
#

from pyscf import gto
from pyscf import scf
from pyscf import soscf

'''
Second order SCF algorithm by decorating the scf object with .newton method.

Second order SCF method need orthonormal orbitals and the corresponding
occupancy as the initial guess.
'''

mol = gto.Mole()
mol.build(
    verbose = 0,
    atom = '''8  0  0.     0
              1  0  -0.757 0.587
              1  0  0.757  0.587''',
    basis = 'ccpvdz',
)

mf = scf.RHF(mol)
mf.conv_tol = 1e-1
mf.kernel()
mo_init = mf.mo_coeff
mocc_init = mf.mo_occ

mf = scf.RHF(mol).newton()
energy = mf.kernel(mo_init, mocc_init)
print('E = %.12f, ref = -76.026765672992' % energy)

mf = scf.UKS(mol).newton()  # Using stream style
# The newton algorithm will automatically generate initial orbitals if initial
# guess is not given.
energy = mf.kernel()
print('E = %.12f, ref = -75.854702461713' % energy)



# Note You should first set mf.xc then apply newton method because this will
# correctly set up the underlying orbital gradients.  If you first create
# mf = mf.newton() then change mf.xc, the orbital Hessian will be computed
# with the updated xc functional while the orbital gradients are computed with
# the old xc functional.
# In some scenario, you can use this character to approximate the
# orbital Hessian, ie, computing the orbital Hessian with approximated XC
# functional if the accurate Hessian is not applicable.
mf = scf.UKS(mol)
mf.xc = 'pbe,pbe'
mf = mf.newton()
energy = mf.kernel()
print('E = %.12f, ref(PBE) = -76.333457658990' % energy)

mf = scf.UKS(mol)
mf = mf.newton()
mf.xc = 'pbe,pbe'
energy = mf.kernel()
print('E = %.12f, ref(LDA) = -75.854702461713' % energy)

#
# (In experiment) Dual-basis can be used in SOSCF object to approximate the
# orbital hessian.
#
mol = gto.M(
    atom = '''8  0  0.     0
              1  0  -0.757 0.587
              1  0  0.757  0.587''',
    basis = 'ccpvtz',
)
mf = scf.RHF(mol).newton()
mf.verbose = 4
# The second basis (for orital hessian) can be assigned to the SOSCF object.
# The orbital gradients will be computed with the main basis which was defined
# in mf._scf object.
mf.mol = soscf.newton_ah.project_mol(mf.mol, dual_basis='ccpvdz')
mf.kernel()

E = -76.026765673101, ref = -76.026765672992
E = -75.854702461713, ref = -75.854702461713
E = -76.333457658990, ref(PBE) = -76.333457658990
E = -75.854702461701, ref(LDA) = -75.854702461713




******** <class 'pyscf.soscf.newton_ah.SecondOrderRHF'> ********
method = SecondOrderRHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /var/folders/q1/krfdq3k10jlff3t1x7mqtv6r0000gn/T/tmpaxgor5ml
max_memory 4000 MB (current use 0 MB)
******** <class 'pyscf.scf.hf.RHF'> Newton solver flags ********
SCF tol = 1e-09
conv_tol_grad = None
max. SCF cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /var/folders/q1/krfdq3k10jlff3t1x7mqtv6r0000gn/T/tmpaxgor5ml
max_cycle_inner = 12
max_stepsize = 0.05
ah_start_tol = 1e+09
ah_level_shift

np.float64(-76.05711408310785)

## WOrking code

In [2]:
from pyscf import gto, scf, mcscf

mol = gto.Mole()
mol.atom = 'Fe 0 0 0; S 0 0 1.826'
mol.basis = basis_dict   # Example basis; customize as needed
mol.spin = 4             # 4 unpaired electrons (quintet state)
mol.charge = 0           # Neutral
mol.build()

print("Total electrons:", mol.nelectron)
print("Number of basis functions (orbitals):", mol.nao_nr())

mf = scf.UHF(mol)
hf_energy = mf.kernel()
print("HF energy:", hf_energy)

norb_active = 6      # Number of active orbitals
nelec_active = 6     # Number of active electrons

mycas = mcscf.CASSCF(mf, norb_active, nelec_active)

#mycas.fcisolver.nroots=1

 
 
#correct answer 
 
# Compute the CASCI/CASSCF energy
e_casci = mycas.kernel()
 
# Output the results
#print("CASCI/CASSCF Ground State Energy:", e_casci[0])
#print("Hartree-Fock Ground State Energy:", hf_energy)


Total electrons: 42
Number of basis functions (orbitals): 27
converged SCF energy = -1655.1015537099  <S^2> = 6.0386361  2S+1 = 5.0154306
HF energy: -1655.101553709904
CASSCF energy = -1655.37432216382
CASCI E = -1655.37432216382  E(CI) = -11.1010881921723  S^2 = 6.0000000


In [25]:
e_casci[0]  # This will give you the total energy of the CASCI/CASSCF calculation

np.float64(-1655.374322243798)

In [19]:
A_code_CASSCF=-1655.37432225459
A_code_CASCI= -1655.37359957703

e_casci[0] - A_code_CASSCF  # This will give you the difference from the reference value

np.float64(7.161816029110923e-08)